# Open source quantum development with Codespaces

This notebook walks through a simple demonstration of how to go from user developed quantum programs to submitting them to be run on cloud resources, all from Python and Jupyter Notebooks. 

All the pieces of the demo are connected by [QIR], or quantum intermediate representation, an open specification maintained by the [QIR Alliance].
QIR is related to LLVM in that it re-uses a lot of the LLVM spec and infrastructure so most of the standard optimizations and utilities work out of the box for quantum programs.

**About the demo**

The basic workflow starts in [QuTiP], a Python package that allows you to express quantum programs in a convinent way.
From there, we will compile the program to QIR from a buil-in QuTiP function that using [PyQIR], a QIR Alliance project.
Once programs are compiled to [QIR], we will us [qat] to run optimization passes on the QIR bytecode.
Lastly, we will send the QIR program to the Azure Quantum service to be executed on quantum hardware.

<INSERT STACK DIAGRAM with explicit tools>

*More details about the repo structrure and contents can be found in the [README](README.md).*

---

## Develop quantum programs in Python

There are a wide variety of languages and frameworks that the OSS community is using to developing quantum progams.
Many of them either directly or via other OSS tools can compile to QIR, you can see a list of options in the [README](README.md) of this repo.
Here we will use [QuTiP](https://qutip.org), the open source quantum toolbox for Python.

> Learn more about quantum programming with the [QuTiP tutorials](https://qutip.org/tutorials#quantum-information-processing)

Lets write a simple teleportation quantum program.

In [1]:
from qutip_qip.circuit import QubitCircuit

# Ask device for some qubits, and create a circuit to manipulate them
circuit: QubitCircuit = QubitCircuit(3, num_cbits=2)
msg, here, there = range(3)

# Encode message
circuit.add_gate("RZ", targets=[msg], arg_value=0.123)

# Entangle qubits
circuit.add_gate("SNOT", targets=[here])
circuit.add_gate("CNOT", targets=[there], controls=[here])
circuit.add_gate("CNOT", targets=[here], controls=[msg])
circuit.add_gate("SNOT", targets=[msg])

# Measure qubits
circuit.add_measurement("Z", targets=[msg], classical_store=0)
circuit.add_measurement("Z", targets=[here], classical_store=1)

# Make final operations based on measurement results
circuit.add_gate("X", targets=[there], classical_controls=[0])
circuit.add_gate("Z", targets=[there], classical_controls=[1])

# Inspect the quantum circuit
circuit.gates

[Gate(RZ, targets=[0], controls=None, classical controls=None, control_value=None, classical_control_value=None),
 Gate(SNOT, targets=[1], controls=None, classical controls=None, control_value=None, classical_control_value=None),
 Gate(CNOT, targets=[2], controls=[1], classical controls=None, control_value=1, classical_control_value=None),
 Gate(CNOT, targets=[1], controls=[0], classical controls=None, control_value=1, classical_control_value=None),
 Gate(SNOT, targets=[0], controls=None, classical controls=None, control_value=None, classical_control_value=None),
 Measurement(Z, target=[0], classical_store=0),
 Measurement(Z, target=[1], classical_store=1),
 Gate(X, targets=[2], controls=None, classical controls=[0], control_value=None, classical_control_value=1),
 Gate(Z, targets=[2], controls=None, classical controls=[1], control_value=None, classical_control_value=1)]

---

## Compile programs to QIR

For more info on the QIR export in QuTiP, check out the [docs](https://qutip-qip.readthedocs.io/en/latest/qip-simulator.html?highlight=qir#import-and-export-quantum-circuits).

In [2]:
from qutip_qip.qir import circuit_to_qir

In [3]:
with open('output/qutip_program.bc', 'wb') as file:
    file.write(circuit_to_qir(circuit, "bitcode"))
    
with open('output/qutip_program.ll', 'w') as file:
    file.write(circuit_to_qir(circuit, "text"))

Great! we now have a way of lowering high-level user code to QIR, but what does that look like?
Let's check out part of the QIR file we just generated:

In [4]:
with open('output/qutip_program.ll', 'r') as f:
    print(f.read())

; ModuleID = 'qutip_circuit'
source_filename = "qutip_circuit"

%Result = type opaque
%Qubit = type opaque

declare void @__quantum__rt__array_start_record_output()

declare void @__quantum__rt__array_end_record_output()

declare void @__quantum__rt__result_record_output(%Result*)

define void @main() #0 {
entry:
  call void @__quantum__qis__rz__body(double 1.230000e-01, %Qubit* null)
  call void @__quantum__qis__h__body(%Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__cnot__body(%Qubit* inttoptr (i64 1 to %Qubit*), %Qubit* inttoptr (i64 2 to %Qubit*))
  call void @__quantum__qis__cnot__body(%Qubit* null, %Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__h__body(%Qubit* null)
  call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  call void @__quantum__qis__mz__body(%Qubit* inttoptr (i64 1 to %Qubit*), %Result* inttoptr (i64 1 to %Result*))
  %0 = call i1 @__quantum__qis__read_result__body(%Result* null)
  br i1 %0, label %then, label %else

In [2]:
import pyqir

In [3]:
pyqir.__dict__

{'__name__': 'pyqir',
 '__doc__': None,
 '__package__': 'pyqir',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7fd7e8173d60>,
 '__spec__': ModuleSpec(name='pyqir', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fd7e8173d60>, origin='/opt/conda/envs/qir-demo/lib/python3.9/site-packages/pyqir/__init__.py', submodule_search_locations=['/opt/conda/envs/qir-demo/lib/python3.9/site-packages/pyqir']),
 '__path__': ['/opt/conda/envs/qir-demo/lib/python3.9/site-packages/pyqir'],
 '__file__': '/opt/conda/envs/qir-demo/lib/python3.9/site-packages/pyqir/__init__.py',
 '__cached__': '/opt/conda/envs/qir-demo/lib/python3.9/site-packages/pyqir/__pycache__/__init__.cpython-39.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name

---

## Optimize programs

This isn't the nicest way to debug things (though really it should be only the compiler looking). Let's look at a control flow graph:

In [5]:
!opt -dot-cfg output/qutip_program.ll -o output/qutip_program.dot && mv .main.dot output/qutip_program.dot

Writing '.main.dot'...


In [136]:
!dot -Tpng output/qutip_program.dot -o output/qutip_program.png

In [11]:
! qat --apply --target-def ./qat_config.yml -S output/qutip_program.ll -o output/qutip_program_optimized.ll

In [12]:
import os

from numpy import size
print(f'Before: {os.stat("output/qutip_program.ll").st_size}, {os.stat("output/qutip_program_optimized.ll").st_size}')

Before: 2520, 2192


---

## Submitting quantum programs to the cloud

Now that we have generated our qir file, we want to submit it to a cloud provider to run it against a simulator or hardware device.

We start by importing the appropriate Python package to work connect to an Azure Quantum resource.

In [13]:
from azure.identity import AzureCliCredential

NOTE: You will have to open a terminal and run `az login` to make sure your Azure credientials are stored.

In [14]:
import azure.quantum
workspace = azure.quantum.Workspace(
    subscription_id="bd1621e9-9094-48e4-99f4-00256be51682",
    resource_group="AzureQuantum",
    name="qir-demo",
    location="eastus",
    credential=AzureCliCredential()
)

In [19]:
with open('./output/qutip_program_simple.bc', 'rb') as f:
    bitcode = f.read()

In [20]:
job = azure.quantum.Job.from_input_data(
    workspace=workspace,
    name="qir-demo",
    provider_id="rigetti",
    target="rigetti.sim.qvm",
    input_data_format="qir.v1",
    output_data_format="microsoft.quantum-results.v1",
    input_data=bitcode,
    input_params={
        "shots": 500,
        "entryPoint": "main",
        "arguments": []
    }
)

In [21]:
info = job.details.as_dict()

In [22]:
job.get_results()

..........